In [1]:
from transformers import AutoTokenizer
import torch
from transformers import AutoModelForSequenceClassification,TrainingArguments,Trainer,EvalPrediction

d:\End_To_End_Project\NLP\Text_Classification\bertvenv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
from datasets import load_from_disk, DatasetDict

In [29]:
dataset_dict = load_from_disk(r"D:\End_To_End_Project\NLP\Text_Classification\artifacts\data_transformation")
tiny_data = DatasetDict()
tiny_data['train'] = dataset_dict['train'].shuffle(seed=1).select(range(100))
tiny_data['validation'] = dataset_dict['validation'].shuffle(seed=1).select(range(40))
tiny_data['test'] = dataset_dict['test'].shuffle(seed=1).select(range(40))

In [35]:
import pandas as pd
df=pd.DataFrame(tiny_data['validation'])

X=df['text']
y=df['label']
df.head()

,text,label
0,The whole purpose of democracy is to create a ...,0
1,Many cities are experiencing perhaps the bigge...,0
2,Many cities around the world have already lear...,0
3,The Electoral College and the Future of Presid...,1
4,Many people around the world drive vehicles to...,0


In [38]:
tiny_data['test']["text"]

['The Importance of Retaining the Electoral College  Dear State Senator\'s Name,  I am writing to express my support for retaining the Electoral College in the election of the President of the United States. The Electoral College, as established by our founding fathers, is a compromise between direct election by popular vote and election by Congress. Although many argue for changing to a popular vote system, I believe that keeping the Electoral College is essential for several reasons.  Firstly, the Electoral College ensures the certainty of outcome. In an election where disputes can arise, the winner of the Electoral College vote consistently achieves a greater share of electoral votes than their share of popular votes. This reduces the likelihood of a disputed outcome, as demonstrated by the 2012 election where President Obama received a higher proportion of electoral votes compared to his popular vote share.  Secondly, the Electoral College promotes the concept of "everyone\'s presi

In [9]:
model_path= r"D:\End_To_End_Project\NLP\Text_Classification\artifacts\model_training\distilbert-base-uncased-finetuned"
tokenizer = AutoTokenizer.from_pretrained(model_path)
model = (AutoModelForSequenceClassification.from_pretrained(model_path))

In [41]:
import torch
import torch.nn.functional as F
from transformers import DistilBertTokenizer, DistilBertForSequenceClassification
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score

# Load tokenizer and model
model_path= r"D:\End_To_End_Project\NLP\Text_Classification\artifacts\model_training\distilbert-base-uncased-finetuned"
tokenizer = DistilBertTokenizer.from_pretrained(model_path)
model = DistilBertForSequenceClassification.from_pretrained(model_path)

# Load evaluation dataset
eval_texts = [...]  # List of evaluation text samples
eval_labels = [...]  # List of evaluation labels

# Tokenize evaluation texts
eval_encodings = tokenizer(tiny_data['test'], truncation=True, padding=True)

# Convert lists to tensors
eval_input_ids = torch.tensor(eval_encodings['input_ids'])
eval_attention_mask = torch.tensor(eval_encodings['attention_mask'])
eval_labels = torch.tensor(eval_labels)

# Evaluate model
with torch.no_grad():
    outputs = model(input_ids=eval_input_ids, attention_mask=eval_attention_mask)

logits = outputs.logits
predictions = torch.argmax(F.softmax(logits, dim=1), dim=1)

# Convert tensors to numpy arrays
eval_labels = eval_labels.numpy()
predictions = predictions.numpy()

# Calculate evaluation metrics
accuracy = accuracy_score(eval_labels, predictions)
precision = precision_score(eval_labels, predictions)
recall = recall_score(eval_labels, predictions)
f1 = f1_score(eval_labels, predictions)
roc_auc = roc_auc_score(eval_labels, logits[:, 1])  # For binary classification

# Print evaluation metrics
print(f'Accuracy: {accuracy:.4f}')
print(f'Precision: {precision:.4f}')
print(f'Recall: {recall:.4f}')
print(f'F1-score: {f1:.4f}')
print(f'ROC AUC: {roc_auc:.4f}')


ValueError: text input must be of type `str` (single example), `List[str]` (batch or single pretokenized example) or `List[List[str]]` (batch of pretokenized examples).

In [10]:
def tokenize_function(batch):  
      
    return tokenizer(batch["text"], padding=True, truncation=True)

In [11]:
data_encoded = tiny_data.map(tokenize_function, batched=True, batch_size=None)


Map: 100%|██████████| 5/5 [00:00<00:00, 66.62 examples/s]


In [17]:
data_encoded

DatasetDict({
    train: Dataset({
        features: ['text', 'label', 'input_ids', 'attention_mask'],
        num_rows: 10
    })
    validation: Dataset({
        features: ['text', 'label', 'input_ids', 'attention_mask'],
        num_rows: 5
    })
    test: Dataset({
        features: ['text', 'label', 'input_ids', 'attention_mask'],
        num_rows: 5
    })
})

In [ ]:
from sklearn.model_selection import train_test_split
X_test, X_holdout, y_test, y_holdout = train_test_split(X_val, y_val, test_size=0.5, random_state=42)

In [33]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score

# Load DistilBERT model and tokenizer
model_name = r"D:\End_To_End_Project\NLP\Text_Classification\artifacts\model_training\distilbert-base-uncased-finetuned"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=2)  # Replace 2 with your number of classes

# Load your dataset
# ... (your data loading and preprocessing code)

# Split data into training, validation, and test sets
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)
X_test, X_holdout, y_test, y_holdout = train_test_split(X_val, y_val, test_size=0.5, random_state=42)

# Fine-tune the model
# ... (your fine-tuning code)

# Evaluate on validation set
predictions_val = model(tokenizer(X_val, padding=True, truncation=True, return_tensors="pt"))["logits"].argmax(dim=-1)
accuracy_val = accuracy_score(y_val, predictions_val)
precision_val = precision_score(y_val, predictions_val)
recall_val = recall_score(y_val, predictions_val)
f1_val = f1_score(y_val, predictions_val)
auc_val = roc_auc_score(y_val, predictions_val)

print(f"Validation Accuracy: {accuracy_val:.4f}")
print


ValueError: text input must be of type `str` (single example), `List[str]` (batch or single pretokenized example) or `List[List[str]]` (batch of pretokenized examples).

In [43]:
df = pd.DataFrame([{"Score" : 123}])
df

,Score
0,123


In [44]:
Dic = {'eval_loss': 0.009427891112864017,
 'eval_accuracy': 0.9966666666666667,
 'eval_runtime': 5.0913,
 'eval_samples_per_second': 58.924,
 'eval_steps_per_second': 3.732,
 'epoch': 3.0}

In [47]:
df =pd.DataFrame([Dic])
df

,eval_loss,eval_accuracy,eval_runtime,eval_samples_per_second,eval_steps_per_second,epoch
0,0.009428,0.996667,5.0913,58.924,3.732,3.0


In [19]:
import numpy as np
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score
from transformers import DistilBertTokenizer, TFDistilBertForSequenceClassification

# Load tokenizer and model
model_path= r"D:\End_To_End_Project\NLP\Text_Classification\artifacts\model_training\distilbert-base-uncased-finetuned"
tokenizer = DistilBertTokenizer.from_pretrained(model_path)
model = TFDistilBertForSequenceClassification.from_pretrained(model_path)


# Load evaluation dataset
eval_texts = tiny_data['validation']["text"] # List of evaluation text samples
eval_labels = tiny_data['validation']["label"] # List of evaluation labels

# Tokenize evaluation texts
eval_encodings = tokenizer(eval_texts, truncation=True, padding=True, return_tensors='pt')

# Convert labels to numpy array
eval_labels = np.array(eval_labels)

# Evaluate model
eval_results = model.predict(eval_encodings)
predictions = np.argmax(eval_results.logits, axis=1)

# Calculate evaluation metrics
accuracy = accuracy_score(eval_labels, predictions)
precision = precision_score(eval_labels, predictions)
recall = recall_score(eval_labels, predictions)
f1 = f1_score(eval_labels, predictions)
roc_auc = roc_auc_score(eval_labels, eval_results.logits[:, 1])  # For binary classification

# Print evaluation metrics
print(f'Accuracy: {accuracy:.4f}')
print(f'Precision: {precision:.4f}')
print(f'Recall: {recall:.4f}')
print(f'F1-score: {f1:.4f}')
print(f'ROC AUC: {roc_auc:.4f}')


ImportError: 
TFDistilBertForSequenceClassification requires the TensorFlow library but it was not found in your environment.
However, we were able to find a PyTorch installation. PyTorch classes do not begin
with "TF", but are otherwise identically named to our TF classes.
If you want to use PyTorch, please use those classes instead!

If you really do want to use TensorFlow, please follow the instructions on the
installation page https://www.tensorflow.org/install that match your environment.
